# OpenAI Compatible API

SGLang provides an OpenAI compatible API for smooth transition from OpenAI services. Full reference of the API is available at [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial aims at these popular APIs:

- `chat/completions`
- `completions`
- `batches`
- `embeddings`(refer to [embedding_model.ipynb](embedding_model.ipynb))

## Chat Completions

### Usage

Similar to [send_request.ipynb](send_request.ipynb), we can send a chat completion request to SGLang server with OpenAI API format.

In [1]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

server_process = execute_shell_command(
    command="python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30000 --host 0.0.0.0"
)

wait_for_server("http://localhost:30000")

[2024-10-30 07:42:27] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, is_embedding=False, host='0.0.0.0', port=30000, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=991357544, constrained_json_whitespace_pattern=None, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, api_key=None, file_storage_pth='SGLang_storage', enable_cache_report=False, watchdog_timeout=600, dp_size=1, load_balance_method='round_robin', dist_init_addr=None, nnodes=1,

[2024-10-30 07:42:43 TP0] Init torch distributed begin.


[2024-10-30 07:42:45 TP0] Load weight begin. avail mem=78.59 GB


[2024-10-30 07:42:45 TP0] lm_eval is not installed, GPTQ may not be usable


INFO 10-30 07:42:45 weight_utils.py:243] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.19it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.59it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.45it/s]

[2024-10-30 07:42:48 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=63.50 GB
[2024-10-30 07:42:48 TP0] Memory pool end. avail mem=8.37 GB
[2024-10-30 07:42:48 TP0] Capture cuda graph begin. This can take up to several minutes.


[2024-10-30 07:42:56 TP0] max_total_num_tokens=442913, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072


[2024-10-30 07:42:56] INFO:     Started server process [1241772]
[2024-10-30 07:42:56] INFO:     Waiting for application startup.
[2024-10-30 07:42:56] INFO:     Application startup complete.
[2024-10-30 07:42:56] INFO:     Uvicorn running on http://0.0.0.0:30000 (Press CTRL+C to quit)


[2024-10-30 07:42:56] INFO:     127.0.0.1:50042 - "GET /v1/models HTTP/1.1" 200 OK


In [2]:
import openai

# Always assign an api_key, even if not specified during server initialization.
# Setting an API key during server initialization is strongly recommended.

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# Chat completion example

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant"},
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2024-10-30 07:42:57 TP0] Prefill batch. #new-seq: 1, #new-token: 49, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-10-30 07:42:57] INFO:     127.0.0.1:50062 - "GET /get_model_info HTTP/1.1" 200 OK
[2024-10-30 07:42:57 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 1, cache hit rate: 1.79%, token usage: 0.00, #running-req: 1, #queue-req: 0
[2024-10-30 07:42:57] INFO:     127.0.0.1:50074 - "POST /generate HTTP/1.1" 200 OK
[2024-10-30 07:42:57] The server is fired up and ready to roll!
[2024-10-30 07:42:57 TP0] Decode batch. #running-req: 1, #token: 89, token usage: 0.00, gen throughput (token/s): 30.33, #queue-req: 0
[2024-10-30 07:42:57] INFO:     127.0.0.1:50048 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=100,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    stop=["\n\n"],  # Simple stop sequence
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

print_highlight(f"Response: {response}")

[2024-10-30 07:42:57 TP0] Prefill batch. #new-seq: 1, #new-token: 48, #cached-token: 28, cache hit rate: 21.97%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-10-30 07:42:57] INFO:     127.0.0.1:50048 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Completions

### Usage

Completions API is similar to Chat Completions API, but without the `messages` parameter.

In [4]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

print_highlight(f"Response: {response}")

[2024-10-30 07:42:57 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, cache hit rate: 21.28%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-10-30 07:42:58 TP0] Decode batch. #running-req: 1, #token: 37, token usage: 0.00, gen throughput (token/s): 106.45, #queue-req: 0


[2024-10-30 07:42:58] INFO:     127.0.0.1:50048 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [5]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

print_highlight(f"Response: {response}")

[2024-10-30 07:42:58 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 1, cache hit rate: 20.53%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-10-30 07:42:58 TP0] Decode batch. #running-req: 1, #token: 15, token usage: 0.00, gen throughput (token/s): 130.82, #queue-req: 0


[2024-10-30 07:42:58 TP0] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, gen throughput (token/s): 134.93, #queue-req: 0


[2024-10-30 07:42:58 TP0] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, gen throughput (token/s): 133.98, #queue-req: 0


[2024-10-30 07:42:59 TP0] Decode batch. #running-req: 1, #token: 135, token usage: 0.00, gen throughput (token/s): 132.86, #queue-req: 0
[2024-10-30 07:42:59] INFO:     127.0.0.1:50048 - "POST /v1/completions HTTP/1.1" 200 OK


## Batches

We have implemented the batches API for chat completions and completions. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [6]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Batch job created with ID: {batch_response.id}")

[2024-10-30 07:42:59] INFO:     127.0.0.1:36722 - "POST /v1/files HTTP/1.1" 200 OK
[2024-10-30 07:42:59] INFO:     127.0.0.1:36722 - "POST /v1/batches HTTP/1.1" 200 OK
[2024-10-30 07:42:59 TP0] Prefill batch. #new-seq: 2, #new-token: 30, #cached-token: 50, cache hit rate: 35.06%, token usage: 0.00, #running-req: 0, #queue-req: 0


In [7]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        print_highlight(f"Request {result['custom_id']}:")
        print_highlight(f"Response: {result['response']}")

    print_highlight("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    print_highlight(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        print_highlight(f"Errors: {batch_response.errors}")

[2024-10-30 07:42:59 TP0] Decode batch. #running-req: 1, #token: 65, token usage: 0.00, gen throughput (token/s): 133.30, #queue-req: 0


Batch job status: validating...trying again in 3 seconds...
[2024-10-30 07:43:02] INFO:     127.0.0.1:36722 - "GET /v1/batches/batch_a39a745f-b375-4fec-b270-a5997923bb0f HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2024-10-30 07:43:02] INFO:     127.0.0.1:36722 - "GET /v1/files/backend_result_file-dd515866-d465-44bc-87ef-8c77e1eefc4c/content HTTP/1.1" 200 OK


[2024-10-30 07:43:02] INFO:     127.0.0.1:36722 - "DELETE /v1/files/backend_result_file-dd515866-d465-44bc-87ef-8c77e1eefc4c HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [8]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = []
for i in range(100):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    print_highlight(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    print_highlight(
        f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
    )

    time.sleep(3)

[2024-10-30 07:43:02] INFO:     127.0.0.1:36736 - "POST /v1/files HTTP/1.1" 200 OK
[2024-10-30 07:43:02] INFO:     127.0.0.1:36736 - "POST /v1/batches HTTP/1.1" 200 OK


[2024-10-30 07:43:02 TP0] Prefill batch. #new-seq: 23, #new-token: 690, #cached-token: 575, cache hit rate: 43.85%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-10-30 07:43:02 TP0] Prefill batch. #new-seq: 77, #new-token: 2310, #cached-token: 1925, cache hit rate: 45.04%, token usage: 0.00, #running-req: 23, #queue-req: 0


[2024-10-30 07:43:02 TP0] Decode batch. #running-req: 100, #token: 4725, token usage: 0.01, gen throughput (token/s): 537.57, #queue-req: 0


[2024-10-30 07:43:03 TP0] Decode batch. #running-req: 100, #token: 8725, token usage: 0.02, gen throughput (token/s): 10898.83, #queue-req: 0


[2024-10-30 07:43:03 TP0] Decode batch. #running-req: 100, #token: 12725, token usage: 0.03, gen throughput (token/s): 10641.40, #queue-req: 0


...

[2024-10-30 07:43:12] INFO:     127.0.0.1:35928 - "GET /v1/batches/batch_86fac791-2cff-40a4-890e-9a029957f3cb HTTP/1.1" 200 OK


[2024-10-30 07:43:15] INFO:     127.0.0.1:35928 - "GET /v1/batches/batch_86fac791-2cff-40a4-890e-9a029957f3cb HTTP/1.1" 200 OK


[2024-10-30 07:43:18] INFO:     127.0.0.1:35928 - "GET /v1/batches/batch_86fac791-2cff-40a4-890e-9a029957f3cb HTTP/1.1" 200 OK


[2024-10-30 07:43:21] INFO:     127.0.0.1:35928 - "GET /v1/batches/batch_86fac791-2cff-40a4-890e-9a029957f3cb HTTP/1.1" 200 OK


[2024-10-30 07:43:24] INFO:     127.0.0.1:35928 - "GET /v1/batches/batch_86fac791-2cff-40a4-890e-9a029957f3cb HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [9]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = []
for i in range(500):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

print_highlight(f"Created batch job with ID: {batch_job.id}")
print_highlight(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    print_highlight(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        print_highlight(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    print_highlight("Batch job successfully cancelled")

except Exception as e:
    print_highlight(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            print_highlight("Successfully cleaned up input file")
    except Exception as e:
        print_highlight(f"Error cleaning up: {e}")
        raise e

[2024-10-30 07:43:27] INFO:     127.0.0.1:55946 - "POST /v1/files HTTP/1.1" 200 OK
[2024-10-30 07:43:27] INFO:     127.0.0.1:55946 - "POST /v1/batches HTTP/1.1" 200 OK


[2024-10-30 07:43:27 TP0] Prefill batch. #new-seq: 36, #new-token: 36, #cached-token: 1944, cache hit rate: 58.68%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-10-30 07:43:27 TP0] Prefill batch. #new-seq: 335, #new-token: 8192, #cached-token: 10231, cache hit rate: 56.46%, token usage: 0.01, #running-req: 36, #queue-req: 129


[2024-10-30 07:43:28 TP0] Prefill batch. #new-seq: 130, #new-token: 3872, #cached-token: 3278, cache hit rate: 54.18%, token usage: 0.03, #running-req: 370, #queue-req: 1


[2024-10-30 07:43:29 TP0] Decode batch. #running-req: 500, #token: 34025, token usage: 0.08, gen throughput (token/s): 901.41, #queue-req: 0


[2024-10-30 07:43:29 TP0] Decode batch. #running-req: 500, #token: 54025, token usage: 0.12, gen throughput (token/s): 24039.15, #queue-req: 0


...

[2024-10-30 07:43:35 TP0] Decode batch. #running-req: 500, #token: 174025, token usage: 0.39, gen throughput (token/s): 18655.53, #queue-req: 0


[2024-10-30 07:43:36 TP0] Decode batch. #running-req: 500, #token: 194025, token usage: 0.44, gen throughput (token/s): 17997.97, #queue-req: 0


[2024-10-30 07:43:37] INFO:     127.0.0.1:49990 - "POST /v1/batches/batch_86f5d6be-4c7c-45f4-abb5-d9d5e88f8356/cancel HTTP/1.1" 200 OK


[2024-10-30 07:43:40] INFO:     127.0.0.1:49990 - "GET /v1/batches/batch_86f5d6be-4c7c-45f4-abb5-d9d5e88f8356 HTTP/1.1" 200 OK


[2024-10-30 07:43:40] INFO:     127.0.0.1:49990 - "DELETE /v1/files/backend_input_file-5cf6c3d0-03f7-44d0-a8b1-426709e81208 HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2024-10-30 07:43:40] INFO:     Shutting down
[2024-10-30 07:43:40] INFO:     Waiting for application shutdown.
[2024-10-30 07:43:40] INFO:     Application shutdown complete.
[2024-10-30 07:43:40] INFO:     Finished server process [1241772]
